### Preprocessimg review text and aspect&sentiment

https://cornac.readthedocs.io/en/latest/_modules/cornac/data/sentiment.html

In [1]:

import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk
import numpy as np
import pandas as pd
import re
from collections import defaultdict
import nltk.corpus
from textblob import TextBlob
from nltk.sentiment import SentimentIntensityAnalyzer
import emoji
sia = SentimentIntensityAnalyzer()
from string import punctuation #'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
from nltk.corpus import stopwords
stop = stopwords.words('english')

##  Get top words and select aspect words


In [2]:
review_df=pd.read_csv("data/wine_reviews.csv", header=0)
review_df_10=pd.read_csv("data/reviews_10_all.csv")

In [3]:

def get_common_words(str_all,N_words, print_words=True):
    
    str_all=''.join (w for w in list(str_all) if w not in list(punctuation+','))
    str_all= emoji.replace_emoji(str_all)
    wd_list=pd.Series([w for w in str_all.split() if w not in stop ]).value_counts()
    freq = wd_list[:N_words]    
    rare = [x[0] for x in wd_list.iteritems() if x[1] < 2]
    if print_words:
        print("\nCommon words:\n")
        print(" | ".join(key+": "+str(value) for key, value in freq.items()))
        print("\nRare words:\n")
        print(" | ".join(key+": "+str(value) for key, value in wd_list[-N_words:].items()))
    return list(freq.keys())

    
top_words=get_common_words(review_df['review'].str.cat(sep=' '),200,print_words=False)
top_words_10=get_common_words(review_df_10['Review'].str.cat(sep=' '),200,print_words=False)

In [54]:
#nltk.pos_tag(top_words)
' '.join(':'.join(x) for x in nltk.pos_tag(top_words))

'wine:NN good:JJ medium:NN finish:NN nose:JJ fruit:NN fruity:NN smooth:RB red:JJ nice:JJ oak:NN dry:JJ light:NN easy:JJ taste:NN drink:NN cherry:NN tannins:VBZ notes:NNS well:RB acidity:RB bodied:VBN sweet:JJ dark:JJ full:JJ great:JJ body:NN palate:JJ vanilla:NN plum:NN long:JJ black:JJ balanced:VBN fruits:NNS value:NN like:IN bit:NN drinking:VBG chocolate:NN leather:NN citrus:NN earthy:JJ strong:JJ blackberry:NN high:JJ berries:NNS slightly:RB hint:VBP apple:NN pepper:NN tobacco:NN little:JJ bottle:NN lovely:RB still:RB lemon:JJ bold:JJ quite:JJ price:NN hints:NNS deep:VBP really:RB one:CD acidic:JJ fresh:JJ ruby:NN much:JJ honey:NN spicy:NN complex:JJ slight:NN tannin:NN green:JJ flavours:NN first:RB better:RBR balance:NN time:NN it’s:JJ oaky:JJ spice:NN aroma:NN soft:JJ strawberry:NN excellent:NN rich:JJ flavour:JJ floral:JJ white:JJ colour:NN beautiful:NN raspberry:NN cherries:NNS ripe:VBP alcohol:NN refreshing:NN young:JJ elegant:JJ food:NN pleasant:NN tannic:NN would:MD decent:VB

In [55]:
' '.join(':'.join(x) for x in nltk.pos_tag(top_words_10))


'wine:NN fruit:NN nose:JJ finish:JJ red:JJ acidity:NN good:JJ I:PRP tannins:VBP cherry:JJ Very:NNP oak:MD A:NNP notes:NNS medium:NN palate:NN nice:JJ body:NN well:RB black:JJ dry:JJ taste:NN dark:JJ vanilla:NN light:JJ smooth:PDT The:DT balanced:JJ fruits:NNS great:JJ Medium:NNP bit:NN long:JJ plum:RB Good:JJ This:DT like:JJ sweet:JJ fruity:NN Great:NNP citrus:NN apple:NN Nice:NNP bodied:VBD pepper:IN full:JJ leather:NN drink:NN fresh:JJ lemon:NN chocolate:NN white:JJ one:CD little:JJ Not:RB color:NN blackberry:NN price:NN value:NN high:JJ aromas:NN green:JJ earthy:JJ Light:NNP Dry:NNP ripe:NN tobacco:NN de:IN berries:NNS balance:NN spice:NN raspberry:VBP colour:NN strawberry:NN bottle:VBD hint:JJ easy:JJ drinking:VBG peach:NN soft:JJ pear:NN complex:JJ years:NNS still:RB It:PRP honey:VBZ really:RB rich:JJ time:NN ruby:VB much:JJ hints:NNS flavours:JJ cherries:NNS Fruity:NNP On:IN alcohol:NN Dark:NNP mouth:NN tannin:NN quite:RB Smooth:NNP Pinot:NNP elegant:JJ aroma:NN intensity:NN mine

## Process item review data

In [4]:
def process_reviews_df(df, col):
    df[col]=df[col].astype(str)
    df[col]=df[col].str.lower()
    df[col]=df[col].apply(lambda x: ''.join (w for w in list(x) if w not in list(punctuation+',')))
    df[col]=df[col].apply(lambda x: emoji.replace_emoji(x))
    df[col]=df[col].apply(lambda x:re.sub('[\s]', ' ', x))
    df[col]=df[col].str.replace('bodied','body')
    df[col]=df[col].str.replace('fruits','fruit')
    df[col]=df[col].str.replace('fruity','fruit')
    df[col]=df[col].str.replace('spicy','spice')
    df[col]=df[col].str.replace('sweetness','sweet')
    df[col]=df[col].str.replace('spicy','spice')
    df[col]=df[col].str.replace('tannin','tannins')
    
    #keep stop words to keep negative words like not, few
    #df[col]=df[col].apply(lambda x: ' '.join (wd for wd in nltk.word_tokenize(x) if wd not in stop))
    return df


In [5]:
review_df_user=pd.read_csv("data/wine_reviews.csv", header=0)
process_reviews_df(review_df_user, 'review')
review_df_top10=pd.read_csv("data/reviews_10_all.csv", header=0)
process_reviews_df(review_df_top10, 'Review')

,user_id,item_id,review
0,34297304,1135067,smoky tobacco
1,34297304,7103,easy to drink amarone not really bold for amar...
2,34297304,5927106,chocolate tobacco cherry
3,34297304,18739,prune
4,34297304,77149,light simple amarone
...,...,...,...
72455,58534725,18978,tropical fruit and clean and refreshing taste ...
72456,58534725,3071535,red plum light body low acidity well balanced ...
72457,58534725,1244,ok quality at this price point not complex enough
72458,58534725,24056,pronounced blackberry and pepper notes full bo...


In [7]:
review_df_user.to_csv("data/reviews_processed.csv", index=False)
review_df_top10.to_csv("data/reviews_10_processed.csv", index=False)

In [8]:
#aspects.csv  -- manually selected
aspects=pd.read_csv("data/aspects.csv", header=0)
WordList = list(aspects["WORD"].astype(str))
WordSentiment=['finish','acidity','tannins','body','aromas','price']


## Start to extract aspects and sentiment

In [10]:
#MODE="test" # for performance monotoring and debugging
MODE="prod"


In [11]:
def def_value():
    return []

In [12]:
def extract_aspect_sentiment(df_review, col):
    dic=defaultdict(def_value)
    if MODE=="prod":
        df=df_review.copy()
    else:
        df=df_review.iloc[:1000].copy()
    df[col]='This is '+ df[col]

    df['token']=df[col].apply(lambda x:nltk.word_tokenize(x))
    df['blob']=df[col].apply(lambda x: TextBlob(x).ngrams(2))
    #print(df)
    #nltk.pos_tag(sent)
    counter=0
    for uid,iid,_,tks,blb in df.itertuples(index=False):
        counter=counter+1
        for wd in list(set(WordList) - set(WordSentiment)):
            if wd in tks:
                dic[uid,iid].append((wd,'notable',1))
        for wd in WordSentiment:
            #print(blb)
            pos=[p[0] for p in blb if (p[1] ==wd and nltk.pos_tag(p)[0][1].startswith("JJ") and sia.polarity_scores(p[0])['neg']<=0.5) ] 
            neg=[p[0] for p in blb if (p[1] ==wd and nltk.pos_tag(p)[0][1].startswith("JJ") and sia.polarity_scores(p[0])['neg']>0.5) ] 
            #pos=[p[0] for p in blb if (p[1] ==wd  and sia.polarity_scores(p[0])['neg']<=0.5) ] 
            #neg=[p[0] for p in blb if (p[1] ==wd  and sia.polarity_scores(p[0])['neg']>0.5) ] 
            pos.extend([p[1] for p in blb if (p[0] ==wd  and nltk.pos_tag(p)[1][1].startswith("JJ") and sia.polarity_scores(p[1])['neg']<=0.5) ] )
            neg.extend([p[1] for p in blb if (p[0] ==wd  and nltk.pos_tag(p)[1][1].startswith("JJ") and sia.polarity_scores(p[1])['neg']>0.5) ] )
            #dic[iid,wd]+=(pos-neg)
            if len(pos)>0:
                #dic[iid].append(':'.join([wd,pos[0],'1']))

                dic[uid,iid].append((wd,pos[0],1))
            if len(neg)>0:
                dic[uid,iid].append((wd,neg[0],-1))


        if counter%20000==0: print(counter)
    return dic


In [13]:
review_df_user=pd.read_csv("data/reviews_processed.csv", header=0)
sentiment_dic=extract_aspect_sentiment(review_df, 'review')

20000
40000
60000


In [14]:
with open("data/sentiment_ui.txt", "w",encoding='utf-8',errors='ignore') as f:
    for key,item in sentiment_dic.items():
        if len(item)>0:
            sline=str(key[0])+','+str(key[1])+','+','.join ([':'.join([x,y,str(z)]) for x,y,z in list(set(item))])
            if sline[-1]==',':
                sline=sline[:-1]
            f.write(f"{sline}\n")

In [24]:
review_df_top10=pd.read_csv("data/reviews_10_processed.csv", header=0)

#Remove blank and nan !!!
sentiment_dic_10=extract_aspect_sentiment(review_df_top10, 'Review')

20000
40000
60000
80000
100000
120000
140000
160000
180000
200000
220000
240000
260000
280000
300000
320000
340000
360000
380000
400000
420000


In [27]:
# combine two dictionaries
sentiment_dic_10.update(sentiment_dic)  

with open("data/sentiment_all.txt", "w",encoding='utf-8',errors='ignore') as f:
    for key,item in sentiment_dic_10.items():
        if len(item)>0:
            sline=str(key[0])+','+str(key[1])+','+','.join ([':'.join([x,y,str(z)]) for x,y,z in list(set(item))])
            if sline[-1]==',':
                sline=sline[:-1]
            f.write(f"{sline}\n")
      

In [26]:


import pickle
with open('sentiment_dic_10.pkl', 'wb') as f:
    pickle.dump(sentiment_dic_10, f)

